In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from patsy import dmatrices
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
df = pd.read_csv('./csv파일/데이터셋_4(EPDO1).csv',encoding='cp949')

In [4]:
y=pd.DataFrame(df['EPDO1'])

In [5]:
x=df.drop(['EPDO1'],axis=1)
x=pd.get_dummies(x,drop_first=True)

corolla = pd.concat([x,y],axis=1)

nCar = corolla.shape[0]
nVar = corolla.shape[1]

print(nCar,nVar)

27284 36


---

---

In [6]:
train_x, test_x, train_y, test_y = train_test_split(x,y, train_size = 0.7, test_size = 0.3)

In [7]:
X_train_scaled = train_x
X_test_scaled = test_x
y_train = train_y
y_test = test_y
X=x

# 랜덤포레스트로 모델 생성

## 1. 변수 중요도

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10).fit(X_train_scaled, y_train)
cross_val_score(rf, X_train_scaled, y_train, cv=3)
rf.score(X_test_scaled, y_test)
print(classification_report(y_test, rf.predict(X_test_scaled)))

feature_importance_rf = pd.DataFrame(zip(X.columns.values, rf.feature_importances_))
feature_importance_rf.columns = ['feature', 'importance']
feature_importance_rf.sort_values("importance", ascending=False, inplace=True)

              precision    recall  f1-score   support

           1       0.01      0.01      0.01       147
           2       0.35      0.24      0.29      2040
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        44
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         1
          32       0.65      0.77      0.70      5093
          33       0.00      0.00      0.00        18
          34       0.00      0.00      0.00        70
          35       0.00      0.00      0.00         0
          36       0.00      0.00      0.00         3
          38       0.00      0.00      0.00         2
          40       0.00      0.00      0.00         0
          64       0.00      0.00      0.00        48
          66       0.00      0.00      0.00         2
          69       0.00      0.00      0.00         1
          70       0.00    

### ▶importance 값이 높을 수록 중요도가 큰 것이다.
### ▶아래의 결과는 값이 큰 순서대로 즉, 내림차순 정렬된 것이다.

In [9]:
feature_importance_rf

,feature,importance
28,도로형태_단일로,0.078170
21,차량용도가해자_그룹4,0.072610
34,발생시각_출퇴근시간,0.066881
9,법규위반가해자_그룹2,0.061169
31,도로선형_평지,0.042936
33,발생시각_저녁~자정,0.040024
6,연령가해자_51-60세,0.038760
5,연령가해자_41-50세,0.037604
2,사고유형_횡단중,0.037422
1,사고유형_차도통행중,0.035128


# 2. 반복적 변수 제거 방식을 이용한 변수 선택

In [10]:
from sklearn.feature_selection import RFE

select = RFE(RandomForestClassifier(n_estimators=5, random_state=42), n_features_to_select = 5)
select.fit(X_train_scaled, y_train)

feature_importance_rfe = pd.DataFrame(zip(X.columns.values, select.get_support()))
feature_importance_rfe.columns = ['feature', 'importance']
feature_importance_rfe.sort_values("importance", ascending=False, inplace=True)

### ▶importance가 True이면 중요하다는 것을 뜻한다. 
#### 피해자신체상해주부위_그룹2: 하퇴부, 얼굴, 발
#### 피해자신체상해주부위_그룹5: 배, 머리, 대퇴부
#### 차량용도가해자_그룹4: 승용차, 사업용기타, 비사업용기타, 버스, 렌터카

In [11]:
feature_importance_rfe

,feature,importance
17,피해자신체상해주부위_그룹5,True
21,차량용도가해자_그룹4,True
31,도로선형_평지,True
14,피해자신체상해주부위_그룹2,True
6,연령가해자_51-60세,True
26,도로종류_지방도,False
22,차량용도가해자_그룹5,False
23,도로종류_군도,False
24,도로종류_시도,False
25,도로종류_일반국도,False
